In [8]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import nltk
nltk.download('stopwords') 
nltk.download('wordnet')
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Chike\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Chike\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
word2vec=Word2Vec.load('C:\\Users\\Chike\\Documents\\Projects\\twitter-disaster-prediction\\models\\word_vector_model.bin')

In [3]:
train_df=pd.read_csv("C:\\Users\\Chike\\Documents\\Projects\\twitter-disaster-prediction\\data\\raw\\train.csv")
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [11]:
train_df['clean_text']=train_df['text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
train_df['clean_text']=train_df['clean_text'].apply(lambda x: re.split('http:\/\/.*', str(x))[0])
train_df['clean_text']=train_df['clean_text'].str.replace('\d+', '')
train_df['clean_text']=train_df['clean_text'].str.lower()
train_df['clean_text']=train_df['clean_text'].str.replace('\W', ' ')
train_df['clean_text']=[' '.join([w for w in x.lower().split() if w not in stop]) for x in train_df['clean_text'].tolist()]
train_df['clean_text']=train_df['clean_text'].str.replace('\d+', '')

train_df.clean_text

<ipython-input-11-b6908ce19e96>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  train_df['clean_text']=train_df['clean_text'].str.replace('\d+', '')
<ipython-input-11-b6908ce19e96>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  train_df['clean_text']=train_df['clean_text'].str.replace('\W', ' ')
<ipython-input-11-b6908ce19e96>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  train_df['clean_text']=train_df['clean_text'].str.replace('\d+', '')


0            deeds reason earthquake may allah forgive us
1                   forest fire near la ronge sask canada
2       residents asked shelter place notified officer...
3       people receive wildfires evacuation orders cal...
4       got sent photo ruby alaska smoke wildfires pou...
                              ...                        
7608    two giant cranes holding bridge collapse nearb...
7609    aria_ahrary thetawniest control wild fires cal...
7610                                utc km volcano hawaii
7611    police investigating e bike collided car littl...
7612    latest homes razed northern california wildfir...
Name: clean_text, Length: 7613, dtype: object

In [12]:
 X_train, X_test, y_train, y_test = train_test_split(train_df.clean_text, train_df.target, test_size=0.2, random_state=42)

In [15]:
tokenizer=Tokenizer()
total_tweets = pd.concat([X_train, X_test])
tokenizer.fit_on_texts(total_tweets)

max_length=max([len(s.split()) for s in total_tweets])

vocab_size=len(tokenizer.word_index)+1

X_train_tokens=tokenizer.texts_to_sequences(X_train)
X_test_tokens=tokenizer.texts_to_sequences(X_test)

X_train_pad=pad_sequences(X_train_tokens,maxlen=max_length,padding='post')
X_test_pad=pad_sequences(X_test_tokens,maxlen=max_length,padding='post')

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.layers.embeddings import Embedding

In [17]:
max_length

25

In [18]:
EMBEDDING_DIM=50

In [28]:
from tensorflow.keras.metrics import Recall
from tensorflow.keras.metrics import Precision

In [29]:
model=Sequential()
model.add(Embedding(vocab_size,EMBEDDING_DIM,input_length=max_length))
model.add(GRU(units=32,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation="sigmoid"))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy',Precision(),Recall()])

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 50)            780200    
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                8064      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 788,297
Trainable params: 788,297
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.fit(X_train_pad,y_train,batch_size=128,epochs=120,validation_data=(X_test_pad,y_test),verbose=2)

Epoch 1/120
48/48 - 8s - loss: 0.6843 - accuracy: 0.5644 - precision: 0.4722 - recall: 0.3053 - val_loss: 0.6822 - val_accuracy: 0.5739 - val_precision: 0.4716 - val_recall: 0.0866
Epoch 2/120
48/48 - 1s - loss: 0.6834 - accuracy: 0.5695 - precision: 0.4716 - recall: 0.0575 - val_loss: 0.6828 - val_accuracy: 0.5739 - val_precision: 0.4716 - val_recall: 0.0411
Epoch 3/120
48/48 - 1s - loss: 0.6835 - accuracy: 0.5696 - precision: 0.4717 - recall: 0.0328 - val_loss: 0.6822 - val_accuracy: 0.5739 - val_precision: 0.4725 - val_recall: 0.0271
Epoch 4/120
48/48 - 1s - loss: 0.6836 - accuracy: 0.5696 - precision: 0.4730 - recall: 0.0232 - val_loss: 0.6823 - val_accuracy: 0.5739 - val_precision: 0.4735 - val_recall: 0.0202
Epoch 5/120
48/48 - 1s - loss: 0.6810 - accuracy: 0.5700 - precision: 0.4752 - recall: 0.0181 - val_loss: 0.6494 - val_accuracy: 0.5850 - val_precision: 0.4836 - val_recall: 0.0168
Epoch 6/120
48/48 - 1s - loss: 0.4516 - accuracy: 0.8061 - precision: 0.6845 - recall: 0.0643 -

48/48 - 1s - loss: 0.0354 - accuracy: 0.9805 - precision: 0.9243 - recall: 0.8053 - val_loss: 1.2967 - val_accuracy: 0.7511 - val_precision: 0.9244 - val_recall: 0.8064
Epoch 47/120
48/48 - 2s - loss: 0.0352 - accuracy: 0.9803 - precision: 0.9245 - recall: 0.8076 - val_loss: 1.3472 - val_accuracy: 0.7511 - val_precision: 0.9246 - val_recall: 0.8086
Epoch 48/120
48/48 - 2s - loss: 0.0361 - accuracy: 0.9803 - precision: 0.9248 - recall: 0.8097 - val_loss: 1.3431 - val_accuracy: 0.7472 - val_precision: 0.9249 - val_recall: 0.8107
Epoch 49/120
48/48 - 2s - loss: 0.0343 - accuracy: 0.9810 - precision: 0.9250 - recall: 0.8118 - val_loss: 1.4488 - val_accuracy: 0.7400 - val_precision: 0.9251 - val_recall: 0.8127
Epoch 50/120
48/48 - 2s - loss: 0.0338 - accuracy: 0.9803 - precision: 0.9252 - recall: 0.8138 - val_loss: 1.4370 - val_accuracy: 0.7466 - val_precision: 0.9252 - val_recall: 0.8147
Epoch 51/120
48/48 - 2s - loss: 0.0335 - accuracy: 0.9808 - precision: 0.9253 - recall: 0.8157 - val_lo

Epoch 92/120
48/48 - 1s - loss: 0.0331 - accuracy: 0.9791 - precision: 0.9277 - recall: 0.8594 - val_loss: 1.4561 - val_accuracy: 0.7433 - val_precision: 0.9277 - val_recall: 0.8597
Epoch 93/120
48/48 - 2s - loss: 0.0336 - accuracy: 0.9810 - precision: 0.9277 - recall: 0.8600 - val_loss: 1.4491 - val_accuracy: 0.7406 - val_precision: 0.9276 - val_recall: 0.8603
Epoch 94/120
48/48 - 2s - loss: 0.0326 - accuracy: 0.9800 - precision: 0.9277 - recall: 0.8606 - val_loss: 1.5383 - val_accuracy: 0.7452 - val_precision: 0.9277 - val_recall: 0.8608
Epoch 95/120
48/48 - 2s - loss: 0.0326 - accuracy: 0.9811 - precision: 0.9277 - recall: 0.8612 - val_loss: 1.5124 - val_accuracy: 0.7439 - val_precision: 0.9277 - val_recall: 0.8614
Epoch 96/120
48/48 - 2s - loss: 0.0322 - accuracy: 0.9803 - precision: 0.9277 - recall: 0.8617 - val_loss: 1.5503 - val_accuracy: 0.7433 - val_precision: 0.9277 - val_recall: 0.8620
Epoch 97/120
48/48 - 1s - loss: 0.0323 - accuracy: 0.9805 - precision: 0.9277 - recall: 0.

In [66]:
val_df=pd.read_csv("C:\\Users\\Chike\\Documents\\Projects\\twitter-disaster-prediction\\data\\raw\\test.csv")
val_df

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [67]:
val_df['clean_text']=val_df['text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
val_df['clean_text']=val_df['clean_text'].apply(lambda x: re.split('http:\/\/.*', str(x))[0])
val_df['clean_text']=val_df['clean_text'].str.replace('\d+', '')
val_df['clean_text']=val_df['clean_text'].str.lower()
val_df['clean_text']=val_df['clean_text'].str.replace('\W', ' ')
val_df['clean_text']=[' '.join([w for w in x.lower().split() if w not in stop]) for x in val_df['clean_text'].tolist()]
val_df['clean_text']=val_df['clean_text'].str.replace('\d+', '')

val_df.clean_text

<ipython-input-67-99147990c7c3>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  val_df['clean_text']=val_df['clean_text'].str.replace('\d+', '')
<ipython-input-67-99147990c7c3>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  val_df['clean_text']=val_df['clean_text'].str.replace('\W', ' ')
<ipython-input-67-99147990c7c3>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  val_df['clean_text']=val_df['clean_text'].str.replace('\d+', '')


0                             happened terrible car crash
1       heard earthquake different cities stay safe ev...
2       forest fire spot pond geese fleeing across str...
3                   apocalypse lighting spokane wildfires
4                     typhoon soudelor kills china taiwan
                              ...                        
3258    earthquake safety los angeles ûò safety fasten...
3259    storm ri worse last hurricane city amp others ...
3260                        green line derailment chicago
3261             meg issues hazardous weather outlook hwo
3262    cityofcalgary activated municipal emergency pl...
Name: clean_text, Length: 3263, dtype: object

In [46]:
def predictions(text):
    sequence=tokenizer.texts_to_sequences(text)
        # pad the sequences
    sequence = pad_sequences(sequence, maxlen=max_length,padding='post')
        # get the prediction
    prediction = model.predict(sequence)[0]
    return prediction[0]#, [np.argmax(prediction)]

In [68]:
val_tokens=tokenizer.texts_to_sequences(val_df.clean_text)

val_pad=pad_sequences(val_tokens,maxlen=max_length,padding='post')

In [70]:
val_df['predictions']=model.predict(val_pad)

In [73]:
val_df.sort_values('predictions',ascending=False)

,id,keyword,location,text,clean_text,predictions
1280,4208,drowned,NaN,100s of migrants feared drowned after 700 sque...,migrants feared drowned squeeze onto boat medi...,0.999957
2273,7586,outbreak,NaN,Families to sue over Legionnaires: More than 4...,families sue legionnaires families affected fa...,0.999957
2270,7572,outbreak,NaN,Families to sue over Legionnaires: More than 4...,families sue legionnaires families affected fa...,0.999957
2272,7575,outbreak,Dubai,Families to sue over Legionnaires: More than 4...,families sue legionnaires families affected fa...,0.999957
2267,7562,outbreak,NaN,Families to sue over Legionnaires: More than 4...,families sue legionnaires families affected fa...,0.999957
...,...,...,...,...,...,...
404,1311,bloody,Baltimore,I swear my eyes be bloody red but bitch I feel...,swear eyes bloody red bitch feel amazing,0.000062
1557,5240,fatality,"Santa Cruz, CA",Don't let your #writing suffer a fatality! Lea...,let writing suffer fatality learn seek destroy...,0.000061
409,1325,bloody,Beacon Hills ~ The Glade,BLOODY HELL TEEN WOLF *O* Hayden and Liam are ...,bloody hell teen wolf hayden liam cute think s...,0.000061
430,1397,body%20bag,New York,Louis Vuitton Monogram Sophie Limited Edition ...,louis vuitton monogram sophie limited edition ...,0.000061


In [74]:
val_df['target']=np.where(val_df['predictions']>=0.5,1,0)

In [76]:
val_df.target.value_counts()

0    1945
1    1318
Name: target, dtype: int64

In [79]:
submission_df=val_df[['id','target']]
submission_df

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,0
3260,10868,1
3261,10874,1


In [81]:
submission_df.to_csv("C:\\Users\\Chike\\Documents\\Projects\\twitter-disaster-prediction\\data\\processed\\2.0_tensorflow_embedding_netword.csv",index=False)

In [80]:
!pwd

/c/Users/Chike/Documents/Projects/twitter-disaster-prediction/notebooks
